**2. Assignment- 1.2**

Resources: https://www.bseindia.com/corporates/List_Scrips.aspx,
https://www.bseindia.com/markets/equity/EQReports/StockPrcHistori.aspx?flag=0

For each company in the active equity listed stocks
1. get the last 5 year closing price for each of these. Here is a sample CSV output
2. find a way of parallelizing the downloads
3. identify the top 10 gainers (%) across groups over the last one month, 3 months,
6 months, one year, 2 years, 3 years and 5 years period.
Note: The gain is the closing price on the last day of the period, divided by the
closing price on the first day of the period, minus 1.

In [2]:
%pip install selenium
%pip install webdriver_manager

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [11]:
import requests
import urllib
from bs4 import BeautifulSoup
import re
import io
import gzip
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import glob
import multiprocessing
from multiprocessing import Pool

driver = webdriver.Chrome(ChromeDriverManager().install())
#chrome driver to run the test scripts automation on google chrome

url = "https://www.bseindia.com/corporates/List_Scrips.aspx"
driver.get(url)

#as per the question, it is Equity we have to find out.
segment = "Equity"
status = "Active"

segment_element = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ddSegment"]')
segment_element.send_keys(segment)

status_element = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ddlStatus"]')
status_element.send_keys(status)

#here we find the particular element or text entered in the text filed box of website.
submit = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_btnSubmit"]')
submit.send_keys(Keys.RETURN)

#here the generated data will be downloaded in a seoarete file upon clicking the submit button on website.
download = driver.find_element_by_xpath(
    '/html/body/div[1]/form/div[4]/div/div/div[2]/div/div/div[2]/a/i')
download.click()
time.sleep(10)
driver.quit()

#now upon reading the euity.csv file, this remaining program will return the details of all tehe security numbers.
df_equity = pd.read_csv("D:\\MSIT\\2-YEAR-GIT-REPOS\\Datascience_2019501102\\DADV\\Assignments\\BSE_Handling and scraping,analysis,Vis,Templates\\Equity.csv")
security_numbers = df_equity["Security Code"].tolist()
security_n = df_equity["Security Name"].tolist()
security_names = dict(zip(security_numbers, security_n))

store_cache = {}

#method to get the stock data.
def get_stocks(nums):
    
    def get_date(d,m,y):
        month = driver.find_element_by_xpath('/html/body/div[1]/div/div/select[1]')   
        for opt in month.find_elements_by_tag_name("option"):
            if opt.text == m:
                opt.click()
                break
        year = driver.find_element_by_xpath('/html/body/div[1]/div/div/select[2]')   
        for opt in year.find_elements_by_tag_name("option"):
            if opt.text == y:
                opt.click()
                break
        days = driver.find_element_by_xpath('/html/body/div[1]/table')
        for row in days.find_elements_by_tag_name('tr'):
            for col in row.find_elements_by_tag_name('td'):
                if col.text == d:
                    row.click()
                    return
    #as in the question syas that we have to retrive closing price for last five years. 
    def traverse_stock(security):
        if security in store_cache:
            return None
        print(security, type(security))
        element = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_smartSearch"]')
        #element = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_divStock"]/div[2]/div/div/div[2]')

        element.clear()
        element.send_keys(security)
        element.send_keys(Keys.ENTER)
        time.sleep(10)
        try:
            response = driver.find_element_by_xpath("/html/body/form/div[4]/div/div/div[1]/div/div[3]/div/div/table/tbody/tr[2]/td/div/div[2]/div/div/div[2]/div[1]/table/tbody/tr/td/div/div/div/ul/li")
        except Exception as e:
            pass

        if response.text == "No Match Found":
            return None
        else:
            from_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtFromDate"]')
            from_date.clear()
            from_date.click()
            get_date("31","Dec","2015")
            #here giving the 2015 as from date and 2020 as to data, totally 5 years.
            to_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtToDate"]')
            to_date.clear()
            to_date.click()
            get_date("31","Dec","2020")
            submit = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_btnSubmit"]')
            submit.click()
            time.sleep(5)
            store_stock(security)
            store_cache[security] = True
            download_xpath = '//*[@id = "ContentPlaceHolder1_btnDownload"]/i'
            driver.find_element_by_xpath(download_xpath).click()
            time.sleep(4)
        return security
            
            
    def store_stock(security):
        table = "/html/body/form/div[4]/div/div/div[1]/div/div[4]/div/div/div[1]/div[4]/div[1]/table/tbody/tr/td/div/span/table"
        table = driver.find_element_by_xpath(table)
        elements = table.find_elements_by_tag_name("tr")
        security = str(security_names[int(security)]) + "_" + str(security)+"_stocks" + ".csv"
        with open(security, "w", newline="") as csvfile:
            writer = csv.writer(csvfile)
            
            #here a csv file for every security number will be forming with all the below provided column names.
            heading = ["Date","Close"]
            writer.writerow(heading)
           
            for row in elements[2:]:

                writer.writerow(
                    [data.text for data in row.find_elements_by_tag_name("td")])
            
            
            
        
    print(nums)
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_argument("--headless")
    driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=chromeOptions)
    url = "https://www.bseindia.com/markets/equity/EQReports/StockPrcHistori.aspx?flag=0"
    driver.get(url)
    traverse_stock(str(nums))
    get_stocks(500002)
    driver.quit()     
    




====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [C:\Users\Ranjith\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache
